# Piškorky algoritmus minmax

In [2]:
import numpy as np

In [36]:
import numpy as np

class State:
    """ Zachycení stavu partie
    gameplan - dvourozměrné pole 3x3
             - 0 - prázdné pole
             - 1 - X
             - 2 - O
    player - hráč, který je v partii na tahu

    current_player - hráč, který je na tahu v daném stavu při prohledávání stavového prostoru
    depth          - hloubka prohledávání stavového prostoru
    max_depth      - maximální délka prohledávání
    """

    generated = 0

    def __init__(self, gameplan, player, current_player=None, depth=0, max_depth=3):
        self.gameplan = gameplan
        self.player = player
        if current_player is None:
            self.current_player = player
        else:
            self.current_player = current_player
        self.depth = depth
        self.max_depth = max_depth
        
        State.generated += 1

    def terminal_test(self):
        """ Metoda otestuje aktuální stav a vrátí hodnotu, zda je partie dohraná a případně, kdo je vítěz

            0 - není konečný stav
            1 - vítězí 1
            2 - vítězí 2
        """
        # pravidla ukončení partie - horizontální a vertikální trojice
        for i in range(3):
            if np.array_equal(self.gameplan[i], [1, 1, 1]):
                return 1
            if np.array_equal(self.gameplan[i], [2, 2, 2]):
                return 2
            if np.array_equal(self.gameplan[:, i], [1, 1, 1]):
                return 1
            if np.array_equal(self.gameplan[:, i], [2, 2, 2]):
                return 2

        # pravidla ukončení partie - diagonály
        if np.array_equal(self.gameplan.diagonal(), [1, 1, 1]):
            return 1
        if np.array_equal(self.gameplan.diagonal(), [2, 2, 2]):
            return 2
        if np.array_equal(np.fliplr(self.gameplan).diagonal(), [1, 1, 1]):
            return 1
        if np.array_equal(np.fliplr(self.gameplan).diagonal(), [2, 2, 2]):
            return 2

        return 0

    def utility(self, result):
        """ Metoda vrací ohodnocení aktuálního stavu partie
            0  - remíza
            1  - vítězí hráč, který je na tahu
            -1 - vítězí protihráč
        """
        if result == 0:
            return 0
        elif result == self.player:
            return 1
        else:
            return -1

    def possible_actions(self):
        """ Metoda vrací list možných akcí pro daný stav partie
            Akce je vyjádřena souřadnicemi prázdného hracího pole.
        """
        possible_actions = []
        # Vyhledání prázdných hracích polí
        for i in range(3):
            for j in range(3):
                if self.gameplan[i][j] == 0:
                    possible_actions.append((i, j))
        return possible_actions

    def expand(self, select_action):
        """ Metoda vytvoří nový stav partie tím, že aplikuje vybranou akci na aktuální stav
            Akce je položení 

            V novém stavu bude na tahu protihráč, ale stav se bude hodnotit z pohledu původního hráče
            Zvýší se i hloubka prohledávaného stavového prostoru
        """
        # Kontrola, zda akce není None (pokud je, nebude pokračováno)
        if select_action is None:
            return None
        
        # Kontrola správnosti zadních souřadnic
        if select_action[0] not in range(3) or select_action[1] not in range(3):
            return None

        # hrací pole musí být volné
        if self.gameplan[select_action[0], select_action[1]] != 0:
            return None
        
        new_array = np.copy(self.gameplan)
        new_array[select_action[0], select_action[1]] = self.current_player
        return State(new_array, 
                     self.player, 
                     self.next_current_player(), 
                     self.depth + 1, 
                     max_depth=self.max_depth)

    def minmax(self, strategy="max"):
        """ Metoda vybere z možných akcí pro daný stav partie tu, která odpovídá strategii """

        if self.depth >= self.max_depth:
            return 0, None

        result = self.terminal_test()
        if result != 0:
            return self.utility(result), None

        if strategy == "max":
            selected_utilization_value = float('-inf')
            next_strategy = "min"
        else:
            selected_utilization_value = float('inf')
            next_strategy = "max"

        actions = self.possible_actions()
        selected_action = actions[0] if actions else None

        for action in actions:
            expanded_state = self.expand(action)
            if expanded_state is None:
                continue

            result = expanded_state.terminal_test()

            if result != 0:
                utilization = expanded_state.utility(result)
            elif len(expanded_state.possible_actions()) == 0:
                utilization = 0
            else:
                utilization, _ = expanded_state.minmax(next_strategy)

            if strategy == "max" and utilization > selected_utilization_value:
                selected_utilization_value = utilization
                selected_action = action
            elif strategy == "min" and utilization < selected_utilization_value:
                selected_utilization_value = utilization
                selected_action = action

        return selected_utilization_value, selected_action

    def next_current_player(self):
        """ Metoda vrátí protihrače v procházení stavového prostoru """
        return 3 - self.current_player

    def next_player(self):
        """ Metoda vrátí protihráče """
        return 3 - self.player


# Partie piškvorek

In [37]:
# Vytvoření počátečního stavu partie
    # herní plán je prázdní
    # na tahu je hrač 1
state = State(gameplan=np.array([[0, 0, 0], [0, 0, 0], [0, 0, 0]]),
              player=1, max_depth=3)

In [38]:
# Vypsání počátečního stavu
print(state.gameplan)

[[0 0 0]
 [0 0 0]
 [0 0 0]]


In [44]:
# Sledujte čas a počty generovaných stavů s postupující partií

# Cyklus pro hru, kde hrají proti sobě dvě kopie algoritmu
while True:
    # Kontrola, zda není partie u konce
    if state is None:  # Přidána kontrola, že state není None
        print("Invalid state encountered. Ending the game.")
        break

    game_result = state.terminal_test()
    if game_result != 0:
        print(f"Winner is {game_result} ")
        break

    # Kontrola, zda není remíza
    if len(state.possible_actions()) == 0:
        print("Drawn")
        break

    # tah hráče
    print(f"=====================\nPlayer {state.player}")
    _, player_action = state.minmax("max")
    print(f"Select action: {player_action}")
    
    # Rozšíření stavu podle vybrané akce
    new_state = state.expand(player_action)

    # Kontrola, zda expand() vrátilo platný stav
    if new_state is None:
        print(f"Invalid action: {player_action}, retrying...")
        continue  # Pokud je akce neplatná, pokračujeme dalším pokusem o platnou akci

    # Pokud je nový stav platný, pokračuj v hře
    state = new_state
    print(state.gameplan)
    print(f"Generated states {State.generated}.")
    State.generated = 0

    # přepnutí partie na druhého hráče
    state.player = state.next_player()


Invalid state encountered. Ending the game.


# Úkol
- Do algoritmu přidejte omezeni na maximalní prohledávanou hloubku.
- Zkoušejte různé omezení prohledávání do hloubky.
- Sledujte, jak se mění časy a počty vygenerovaných stavů
- Mění se výsledky hry?

In [48]:
import time
import numpy as np

# Vytvoření počátečního stavu partie
state = State(gameplan=np.array([[0, 0, 0], [0, 0, 0], [0, 0, 0]]),
              player=1, max_depth=2)

# Cyklus pro hru, kde hrají proti sobě dvě kopie algoritmu
while True:
    # Měření času na začátku každého kroku
    start_time = time.time()

    # Kontrola, zda není partie u konce
    game_result = state.terminal_test()
    if game_result != 0:
        print(f"Winner is {game_result} ")
        break

    # Kontrola, zda není remíza
    if len(state.possible_actions()) == 0:
        print("Drawn")
        break

    # tah hráče
    print(f"=====================\nPlayer {state.player}")
    _, player_action = state.minmax("max")
    print(f"Select action: {player_action}")
    
    # Rozšíření stavu podle vybrané akce
    next_state = state.expand(player_action)
    
    # Pokud expand vrátil None, znamená to, že akce není platná, a my ji přeskočíme
    if next_state is None:
        print(f"Invalid action {player_action}. Trying another action.")
        
        # Získání dalších možných akcí a pokusíme se je provést
        possible_actions = state.possible_actions()
        
        if not possible_actions:
            print("No valid actions left. The game is a draw.")
            break
        
        # Vybereme první platnou akci, aby nedošlo k zacyklení
        player_action = possible_actions[0]
        next_state = state.expand(player_action)
    
    state = next_state
    print(state.gameplan)
    print(f"Generated states {State.generated}.")
    
    # Zaznamenání času, jak dlouho trval tah
    elapsed_time = time.time() - start_time
    print(f"Time taken for this move: {elapsed_time:.4f} seconds")
    
    # Resetování generovaných stavů pro tento tah
    State.generated = 0

    # přepnutí partie na druhého hráče
    state.player = state.next_player()


Player 1
Select action: (0, 0)
[[1 0 0]
 [0 0 0]
 [0 0 0]]
Generated states 83.
Time taken for this move: 0.0123 seconds
Player 2
Select action: (0, 1)
[[1 2 0]
 [0 0 0]
 [0 0 0]]
Generated states 9.
Time taken for this move: 0.0015 seconds
Player 1
Select action: None
Invalid action None. Trying another action.
[[1 2 1]
 [0 0 0]
 [0 0 0]]
Generated states 1.
Time taken for this move: 0.0000 seconds
Player 2
Select action: None
Invalid action None. Trying another action.
[[1 2 1]
 [2 0 0]
 [0 0 0]]
Generated states 1.
Time taken for this move: 0.0000 seconds
Player 1
Select action: None
Invalid action None. Trying another action.
[[1 2 1]
 [2 1 0]
 [0 0 0]]
Generated states 1.
Time taken for this move: 0.0000 seconds
Player 2
Select action: None
Invalid action None. Trying another action.
[[1 2 1]
 [2 1 2]
 [0 0 0]]
Generated states 1.
Time taken for this move: 0.0000 seconds
Player 1
Select action: None
Invalid action None. Trying another action.
[[1 2 1]
 [2 1 2]
 [1 0 0]]
Generated 